# TEAM LOSERS: Milestone 3

In [6]:
import numpy as np 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import re
import random
import math
import pandas as pd 
from sklearn import metrics
import matplotlib.pyplot as plt

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn import metrics

## Import Files

In [8]:
o = os.getcwd()

data = pd.read_csv(o + "\\..\\data\\cases_train_processed.csv", parse_dates = True)
data_test = pd.read_csv(o + "\\..\\data\\cases_test_processed.csv", parse_dates=True)
plot_path = o + "\\..\\plots\\"
results_path = o + "\\..\\results\\"

'''
# For Mac/Linux
data = pd.read_csv(o + "/../data/cases_train_processed.csv", parse_dates = True)
data_test = pd.read_csv(o + "/../data/cases_test_processed.csv", parse_dates = True)
plot_path = o + "/../plots/"
results_path = o + "/../results/"
'''

'\n# For Mac/Linux\ndata = pd.read_csv(o + "/../data/cases_train_processed.csv", parse_dates = True)\ndata_test = pd.read_csv(o + "/../data/cases_test_processed.csv", parse_dates = True)\nplot_path = o + "/../plots/"\nresults_path = o + "/../results/"\n'

## Categorize Functions


In [9]:
# converts string Series into a cataegorized int series for data analysis
def categorize_column(data):
    i = 0
    for value in tqdm(data.unique()):
        data.replace(value, i, inplace = True)
        i += 1
    data = data.apply(pd.to_numeric)
    return data

# converts string Series into a cataegorized int series1 for data analysis
def categorize_outcome(data):
    data = data.map({'nonhospitalized':0, 'deceased':1, 'recovered':2, 'hospitalized':3},na_action ='ignore')
    data = data.apply(pd.to_numeric)
    return data

## Data Cleanup Functions

In [10]:
def cleanup(data):
    data = data.drop(columns=['province', 'country'])
    data['sex'] = categorize_column(data['sex'])
    data['outcome'] = categorize_outcome(data['outcome'])
    data['Combined_Key'] = categorize_column(data['Combined_Key'])
    return data

## Data Output Functions

In [18]:
def output_predictions(p):
    pred = []
    arr = {0:'nonhospitalized', 1:'deceased',2:'recovered', 3:'hospitalized'}
    for i in p:
        pred.append(arr[i])
    with open(results_path+"predictions.txt", "w") as f:
        f.writelines('\n'.join(pred))
    return pred

In [12]:
data = cleanup(data)
data_test = cleanup(data_test)

100%|██████████| 413/413 [00:04<00:00, 100.57it/s]


## Build and Run KNN Classifier

In [13]:
knn = KNeighborsClassifier(leaf_size = 100, n_neighbors = 4, p=2, weights = 'uniform' )

In [14]:
X = data.drop(columns='outcome')
y = data['outcome']
knn.fit(X,y)

KNeighborsClassifier(leaf_size=100, n_neighbors=4)

## Outputting Text file and Getting Statistics

In [19]:
p = knn.predict(data_test.drop(columns='outcome'))
pred = output_predictions(p)

In [26]:
df = pd.Series( (v for v in pred) )

In [29]:
df.value_counts()

nonhospitalized    20677
recovered          13823
hospitalized       11858
deceased             142
dtype: int64

In [36]:
data_test = pd.read_csv(o + "\\..\\data\\cases_test_processed.csv", parse_dates=True)

In [37]:
data_test.outcome = df

In [40]:
data_test[data_test.outcome=='deceased']

,age,sex,province,country,latitude,longitude,outcome,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
2075,70,male,Hubei,China,30.290590,113.37500,deceased,68139,4512,63627,0,"Hubei, China",115.158019,6.621758
2078,87,male,Hubei,China,30.625060,114.34210,deceased,68139,4512,63627,0,"Hubei, China",115.158019,6.621758
2081,66,female,Hubei,China,30.625060,114.34210,deceased,68139,4512,63627,0,"Hubei, China",115.158019,6.621758
2083,78,male,Hubei,China,30.625060,114.34210,deceased,68139,4512,63627,0,"Hubei, China",115.158019,6.621758
2084,89,male,Hubei,China,30.625060,114.34210,deceased,68139,4512,63627,0,"Hubei, China",115.158019,6.621758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44793,90,male,Zambales,Philippines,14.927990,120.21081,deceased,283460,4930,209885,68645,Philippines,258.676030,1.739222
46471,74,male,Washington,United States,47.491332,-121.80364,deceased,82199,2037,0,80162,"Washington, US",4488.326763,2.478162
46473,74,female,Washington,United States,47.491332,-121.80364,deceased,82199,2037,0,80162,"Washington, US",4488.326763,2.478162
46477,74,male,Washington,United States,47.491332,-121.80364,deceased,82199,2037,0,80162,"Washington, US",4488.326763,2.478162
